In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df_ratings = pd.read_csv('data/ratings.csv')
df_ratings.head(1)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703


In [3]:
df_ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [4]:
df_ratings["user_id"] = df_ratings["userId"].astype("category").cat.codes
df_ratings["item_id"] = df_ratings["movieId"].astype("category").cat.codes

In [5]:
df_ratings.describe()

,userId,movieId,rating,timestamp,user_id,item_id
count,100836.000000,100836.000000,100836.000000,1.008360e+05,100836.000000,100836.000000
mean,326.127564,19435.295718,3.501557,1.205946e+09,325.127564,3101.735561
std,182.618491,35530.987199,1.042529,2.162610e+08,182.618491,2627.050983
min,1.000000,1.000000,0.500000,8.281246e+08,0.000000,0.000000
25%,177.000000,1199.000000,3.000000,1.019124e+09,176.000000,900.000000
50%,325.000000,2991.000000,3.500000,1.186087e+09,324.000000,2252.000000
75%,477.000000,8122.000000,4.000000,1.435994e+09,476.000000,5095.250000
max,610.000000,193609.000000,5.000000,1.537799e+09,609.000000,9723.000000


In [6]:
df_ratings.astype("category").describe()

,userId,movieId,rating,timestamp,user_id,item_id
count,100836,100836,100836.0,100836,100836,100836
unique,610,9724,10.0,85043,610,9724
top,414,356,4.0,1459787998,413,314
freq,2698,329,26818.0,128,2698,329


In [7]:
df_ratings.head()

,userId,movieId,rating,timestamp,user_id,item_id
0,1,1,4.0,964982703,0,0
1,1,3,4.0,964981247,0,2
2,1,6,4.0,964982224,0,5
3,1,47,5.0,964983815,0,43
4,1,50,5.0,964982931,0,46


In [8]:
train_ratings, test_ratings = train_test_split(df_ratings, test_size=0.1, random_state=42)
train_ratings, val_ratings = train_test_split(train_ratings, test_size=0.1, random_state=42)

In [9]:
class MovieLensDataset(Dataset):
    def __init__(self, ratings):
        self.user_ids = torch.LongTensor(df_ratings['user_id'].values)
        self.item_ids = torch.LongTensor(df_ratings['item_id'].values)
        self.ratings = torch.FloatTensor(df_ratings['rating'].values)
        
    def __len__(self):
        return len(self.user_ids)
    
    def __getitem__(self, idx):
        return self.user_ids[idx], self.item_ids[idx], self.ratings[idx]
        
train_dataset = MovieLensDataset(train_ratings)
val_dataset = MovieLensDataset(val_ratings)
test_dataset = MovieLensDataset(test_ratings)

In [10]:
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [11]:
class RecommenderNet(nn.Module):
    def __init__(self, num_users, num_items, emb_size=64):
        super().__init__()
        # print("num_users", num_users, "num_items", num_items)
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.fc1 = nn.Linear(emb_size*2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        
    def forward(self, user_ids, item_ids):
        user_emb = self.user_emb(user_ids)
        item_emb = self.item_emb(item_ids)
        x = torch.cat([user_emb, item_emb], dim=-1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.relu(x)
        return x.squeeze()

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RecommenderNet(num_users=df_ratings['user_id'].nunique(), num_items=df_ratings['item_id'].nunique())
model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [13]:
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0.
    for user_ids, item_ids, ratings in dataloader:
        user_ids, item_ids, ratings = user_ids.view(-1).to(device), item_ids.view(-1).to(device), ratings.to(device)
        optimizer.zero_grad()
        outputs = model(user_ids, item_ids)
        loss = criterion(outputs, ratings)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.
    with torch.no_grad():
        for user_ids, item_ids, ratings in dataloader:
            user_ids, item_ids, ratings = user_ids.view(-1).to(device), item_ids.view(-1).to(device), ratings.to(device)
            outputs = model(user_ids, item_ids)
            loss = criterion(outputs, ratings)
            total_loss += loss.item()
    return total_loss / len(dataloader)


In [14]:
n_epochs = 10
for epoch in range(n_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss = evaluate(model, val_loader, criterion, device)
    print(f'epoch {epoch+1}, train_loss: {train_loss:.4f}, val_loss: {val_loss:.4f}')

test_loss = evaluate(model, test_loader, criterion, device)
print(f'test_loss: {test_loss:.4f}')

epoch 1, train_loss: 1.5253, val_loss: 0.7986
epoch 2, train_loss: 0.9455, val_loss: 0.7435
epoch 3, train_loss: 0.8319, val_loss: 0.7348
epoch 4, train_loss: 0.7967, val_loss: 0.7032
epoch 5, train_loss: 0.7776, val_loss: 0.6693
epoch 6, train_loss: 0.7850, val_loss: 0.6752
epoch 7, train_loss: 0.7697, val_loss: 0.6616
epoch 8, train_loss: 0.7650, val_loss: 0.6382
epoch 9, train_loss: 0.7682, val_loss: 0.6313
epoch 10, train_loss: 0.7737, val_loss: 0.6564
test_loss: 0.6564
